In [1]:
from transformers import AutoModel, BloomModel

In [2]:
bloom_model = "bigscience/bloom-560m-intermediate"
bloom_checkpoint = "global_step1000"

In [6]:
model = BloomModel.from_pretrained(bloom_model, 
                                    revision=bloom_checkpoint,
                                    torch_dtype="auto",
                                    output_hidden_states=True
                                    )